# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6c84e1a910>
├── 'a' --> tensor([[ 0.2286,  1.7503,  0.5964],
│                   [-0.5357, -1.0325, -0.5516]])
└── 'x' --> <FastTreeValue 0x7f6c84e1aa90>
    └── 'c' --> tensor([[ 1.5782, -1.0918, -0.3631, -0.1462],
                        [-1.4253,  0.4138,  2.3409,  1.2072],
                        [ 1.2934, -0.4755, -2.2869,  0.0230]])

In [4]:
t.a

tensor([[ 0.2286,  1.7503,  0.5964],
        [-0.5357, -1.0325, -0.5516]])

In [5]:
%timeit t.a

75.9 ns ± 0.638 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6c84e1a910>
├── 'a' --> tensor([[ 0.5734,  0.2763,  0.5117],
│                   [ 0.5525, -1.2583, -0.3615]])
└── 'x' --> <FastTreeValue 0x7f6c84e1aa90>
    └── 'c' --> tensor([[ 1.5782, -1.0918, -0.3631, -0.1462],
                        [-1.4253,  0.4138,  2.3409,  1.2072],
                        [ 1.2934, -0.4755, -2.2869,  0.0230]])

In [7]:
%timeit t.a = new_value

84.5 ns ± 1.59 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2286,  1.7503,  0.5964],
               [-0.5357, -1.0325, -0.5516]]),
    x: Batch(
           c: tensor([[ 1.5782, -1.0918, -0.3631, -0.1462],
                      [-1.4253,  0.4138,  2.3409,  1.2072],
                      [ 1.2934, -0.4755, -2.2869,  0.0230]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2286,  1.7503,  0.5964],
        [-0.5357, -1.0325, -0.5516]])

In [11]:
%timeit b.a

73.5 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1256, -1.0470,  0.5039],
               [-1.4103, -1.0691,  0.8078]]),
    x: Batch(
           c: tensor([[ 1.5782, -1.0918, -0.3631, -0.1462],
                      [-1.4253,  0.4138,  2.3409,  1.2072],
                      [ 1.2934, -0.4755, -2.2869,  0.0230]]),
       ),
)

In [13]:
%timeit b.a = new_value

629 ns ± 6.54 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.5 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

290 µs ± 16.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

271 µs ± 16 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6bd7aa8fd0>
├── 'a' --> tensor([[[ 0.2286,  1.7503,  0.5964],
│                    [-0.5357, -1.0325, -0.5516]],
│           
│                   [[ 0.2286,  1.7503,  0.5964],
│                    [-0.5357, -1.0325, -0.5516]],
│           
│                   [[ 0.2286,  1.7503,  0.5964],
│                    [-0.5357, -1.0325, -0.5516]],
│           
│                   [[ 0.2286,  1.7503,  0.5964],
│                    [-0.5357, -1.0325, -0.5516]],
│           
│                   [[ 0.2286,  1.7503,  0.5964],
│                    [-0.5357, -1.0325, -0.5516]],
│           
│                   [[ 0.2286,  1.7503,  0.5964],
│                    [-0.5357, -1.0325, -0.5516]],
│           
│                   [[ 0.2286,  1.7503,  0.5964],
│                    [-0.5357, -1.0325, -0.5516]],
│           
│                   [[ 0.2286,  1.7503,  0.5964],
│                    [-0.5357, -1.0325, -0.5516]]])
└── 'x' --> <FastTreeValue 0x7f6bd7aa8eb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.2 µs ± 697 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6bd7aa8df0>
├── 'a' --> tensor([[ 0.2286,  1.7503,  0.5964],
│                   [-0.5357, -1.0325, -0.5516],
│                   [ 0.2286,  1.7503,  0.5964],
│                   [-0.5357, -1.0325, -0.5516],
│                   [ 0.2286,  1.7503,  0.5964],
│                   [-0.5357, -1.0325, -0.5516],
│                   [ 0.2286,  1.7503,  0.5964],
│                   [-0.5357, -1.0325, -0.5516],
│                   [ 0.2286,  1.7503,  0.5964],
│                   [-0.5357, -1.0325, -0.5516],
│                   [ 0.2286,  1.7503,  0.5964],
│                   [-0.5357, -1.0325, -0.5516],
│                   [ 0.2286,  1.7503,  0.5964],
│                   [-0.5357, -1.0325, -0.5516],
│                   [ 0.2286,  1.7503,  0.5964],
│                   [-0.5357, -1.0325, -0.5516]])
└── 'x' --> <FastTreeValue 0x7f6c60a7a2b0>
    └── 'c' --> tensor([[ 1.5782, -1.0918, -0.3631, -0.1462],
                        [-1.4253,  0.4138,  2.3409,  1.2072],
                 

In [23]:
%timeit t_cat(trees)

46.2 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

82 µs ± 458 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2286,  1.7503,  0.5964],
                [-0.5357, -1.0325, -0.5516]],
       
               [[ 0.2286,  1.7503,  0.5964],
                [-0.5357, -1.0325, -0.5516]],
       
               [[ 0.2286,  1.7503,  0.5964],
                [-0.5357, -1.0325, -0.5516]],
       
               [[ 0.2286,  1.7503,  0.5964],
                [-0.5357, -1.0325, -0.5516]],
       
               [[ 0.2286,  1.7503,  0.5964],
                [-0.5357, -1.0325, -0.5516]],
       
               [[ 0.2286,  1.7503,  0.5964],
                [-0.5357, -1.0325, -0.5516]],
       
               [[ 0.2286,  1.7503,  0.5964],
                [-0.5357, -1.0325, -0.5516]],
       
               [[ 0.2286,  1.7503,  0.5964],
                [-0.5357, -1.0325, -0.5516]]]),
    x: Batch(
           c: tensor([[[ 1.5782, -1.0918, -0.3631, -0.1462],
                       [-1.4253,  0.4138,  2.3409,  1.2072],
                       [ 1.2934, -0.4755, -2.2869,  0.0230]],
         

In [26]:
%timeit Batch.stack(batches)

106 µs ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2286,  1.7503,  0.5964],
               [-0.5357, -1.0325, -0.5516],
               [ 0.2286,  1.7503,  0.5964],
               [-0.5357, -1.0325, -0.5516],
               [ 0.2286,  1.7503,  0.5964],
               [-0.5357, -1.0325, -0.5516],
               [ 0.2286,  1.7503,  0.5964],
               [-0.5357, -1.0325, -0.5516],
               [ 0.2286,  1.7503,  0.5964],
               [-0.5357, -1.0325, -0.5516],
               [ 0.2286,  1.7503,  0.5964],
               [-0.5357, -1.0325, -0.5516],
               [ 0.2286,  1.7503,  0.5964],
               [-0.5357, -1.0325, -0.5516],
               [ 0.2286,  1.7503,  0.5964],
               [-0.5357, -1.0325, -0.5516]]),
    x: Batch(
           c: tensor([[ 1.5782, -1.0918, -0.3631, -0.1462],
                      [-1.4253,  0.4138,  2.3409,  1.2072],
                      [ 1.2934, -0.4755, -2.2869,  0.0230],
                      [ 1.5782, -1.0918, -0.3631, -0.1462],
                      [-1.4253,  

In [28]:
%timeit Batch.cat(batches)

189 µs ± 2.43 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

593 µs ± 41.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
